In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import key_dict

#gmaps key 
g_key = key_dict['g_key']

 Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [6]:
weather_csv_file = "../WeatherPy/output/weather_info_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,80.13,71,0,7.54,PF,1616357944
1,Ushuaia,-54.8000,-68.3000,55.40,58,40,18.41,AR,1616357707
2,Busselton,-33.6500,115.3333,64.99,95,80,3.00,AU,1616357988
3,Sakakah,29.9697,40.2064,73.40,17,0,4.61,SA,1616358327
4,Saldanha,-33.0117,17.9442,62.60,67,0,6.91,ZA,1616358034


 Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [15]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [19]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Clouds"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
3,Sakakah,29.9697,40.2064,73.40,17,0,4.61,SA,1616358327
130,Akaltara,22.0167,82.4333,74.57,30,0,2.19,IN,1616358362
149,Bandarbeyla,9.4942,50.8122,77.79,69,0,7.83,SO,1616358068
163,Kālāgarh Project Colony,29.5000,78.7500,70.66,26,0,5.12,IN,1616358372
205,Mercedes,-34.6515,-59.4307,75.99,61,0,1.01,AR,1616358385
220,Buraidah,26.3260,43.9750,73.40,38,0,2.30,SA,1616358391
251,Ahwa,20.7500,73.6833,72.01,16,0,1.95,IN,1616358401
260,Inhambane,-23.8650,35.3833,76.28,86,0,7.07,MZ,1616358404
267,Dongargaon,20.9667,80.8500,78.66,30,0,3.02,IN,1616358406
278,Salalah,17.0151,54.0924,78.80,89,0,1.14,OM,1616357773


 Hotel Map

Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.